<a href="https://colab.research.google.com/github/francescopandolfo/TESI_LM32/blob/main/RF%20-%20variazione_relativa%20-%20eventi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime

import warnings
warnings.filterwarnings("ignore")
pathDrive = 'drive/MyDrive/UNICT/_TESI/dati/'

#df = pd.read_csv('sample_data/dati_interpolati_diff_1d-2d_undsamp_7gg.csv', sep=';')
df = pd.read_csv(pathDrive + 'dati_interpolati_diff_1d-2d_8gg - eventi.csv', sep=';')
#df = pd.read_csv('tesi/dati_interpolati_diff_1d_undsamp_7gg_2classi.csv', sep=';')
#df = pd.read_csv('sample_data/dati_interpolati_diff_3d-3d_undsamp_7gg.csv', sep=';')

df['edam_ecne_eplu_diff'] = df['edam_ecne_eplu_diff'].str.replace(',','.').astype(float)#
df['edam_ecor_emcn_diff'] = df['edam_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['edam_emsg_eplu_diff'] = df['edam_emsg_eplu_diff'].str.replace(',','.').astype(float)
df['edam_epdn_ecne_diff'] = df['edam_epdn_ecne_diff'].str.replace(',','.').astype(float)
df['edam_epdn_emcn_diff'] = df['edam_epdn_emcn_diff'].str.replace(',','.').astype(float)
df['eint_ecpn_emgl_diff'] = df['eint_ecpn_emgl_diff'].str.replace(',','.').astype(float)#
df['eint_esln_emgl_diff'] = df['eint_esln_emgl_diff'].str.replace(',','.').astype(float)
df['eint_esln_espc_diff'] = df['eint_esln_espc_diff'].str.replace(',','.').astype(float)
df['emeg_ecpn_emgl_diff'] = df['emeg_ecpn_emgl_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_ecri_diff'] = df['emfn_ecor_ecri_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_emcn_diff'] = df['emfn_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['emfn_emcn_epdn_diff'] = df['emfn_emcn_epdn_diff'].str.replace(',','.').astype(float)
df['epdn_ecne_ecpn_diff'] = df['epdn_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['epdn_eint_ecpn_diff'] = df['epdn_eint_ecpn_diff'].str.replace(',','.').astype(float)##
df['epdn_eint_espc_diff'] = df['epdn_eint_espc_diff'].str.replace(',','.').astype(float)
df['epdn_emfn_espc_diff'] = df['epdn_emfn_espc_diff'].str.replace(',','.').astype(float)
df['eplu_ecne_ecpn_diff'] = df['eplu_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_ecpn_diff'] = df['eplu_emeg_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_emal_diff'] = df['eplu_emeg_emal_diff'].str.replace(',','.').astype(float)
df['eplu_emsg_emal_diff'] = df['eplu_emsg_emal_diff'].str.replace(',','.').astype(float)

_dftarget = df['evento'].astype(int).values

format = '%d/%m/%Y'
df['date'] = pd.to_datetime(df['day'], format=format)
df = df.set_index('date')
df = df.drop(columns=['day'])
df = df.drop(columns=['evento'])

for i in reversed(range(len(df.columns))):
  if df.dtypes[df.columns[i]] == 'object':
    df = df.drop(df.columns[i], axis=1)

normalized_df = (df - df.mean()) / df.std()
df = normalized_df

#print(df.info())


In [3]:
from sklearn import datasets, decomposition
pca_feature_number = 14
pca = decomposition.PCA(n_components = pca_feature_number)
pca.fit(df)
new_df = pca.transform(df)

for i in range(pca_feature_number):
  df['f_' + str(i)] = new_df[:,i]

df = df.filter(like='f_')
#print(df.info())

In [5]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_decision_regions
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV #cross validation (k-fold)
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

In [78]:
from sklearn import datasets, decomposition

y = _dftarget
split_at = 98

for a in range(20):
  i=a+1
  pca_feature_number = i
  pca = decomposition.PCA(n_components = pca_feature_number)
  pca.fit(df)
  new_df = pca.transform(df.copy())

  _df = df.copy()

  for j in range(i):
    _df['f_' + str(j)] = new_df[:,j]

  _df = _df.filter(like='f_')


  X = _df.values[: , :]
  who = "pca-" + str(i)

  X_train = X[:split_at]
  X_test = X[split_at:]

  y_train = y[:split_at]
  y_test = y[split_at:]


  lr = LogisticRegressionCV(cv=5, random_state=0, class_weight='balanced') #scoring per cambiare metrica
  lr.fit(X_train, y_train)
  acc_s = lr.score(X_train, y_train)*100
  acc_s_test = lr.score(X_test, y_test)*100

  print( who + ";" + "{:.1f}".format(acc_s) + ";" + "{:.1f}".format(acc_s_test) +";" + "{:.1f}".format(acc_s+acc_s_test) )


pca-1;40.8;42.9;83.7
pca-2;54.1;38.1;92.2
pca-3;56.1;38.1;94.2
pca-4;59.2;50.0;109.2
pca-5;61.2;45.2;106.5
pca-6;58.2;45.2;103.4
pca-7;59.2;42.9;102.0
pca-8;56.1;40.5;96.6
pca-9;60.2;40.5;100.7
pca-10;63.3;42.9;106.1
pca-11;61.2;42.9;104.1
pca-12;63.3;42.9;106.1
pca-13;64.3;42.9;107.1
pca-14;73.5;54.8;128.2
pca-15;75.5;54.8;130.3
pca-16;68.4;42.9;111.2
pca-17;76.5;57.1;133.7
pca-18;66.3;42.9;109.2
pca-19;66.3;42.9;109.2
pca-20;66.3;42.9;109.2


In [ ]:
y = _dftarget
split_at = 98
plottare = False

#max_f1_score = 0
#max_f1_score_who = ""
max_accuracy_score = 0
max_accuracy_score_who = ""

for i in range(pca_feature_number):
  for j in range(pca_feature_number):
    for a in range(pca_feature_number):
      for b in range(pca_feature_number):
        for c in range(pca_feature_number):
          for d in range(pca_feature_number):
            for e in range(pca_feature_number):
              for f in range(pca_feature_number):
                for g in range(pca_feature_number):
                  for h in range(pca_feature_number):

                    if i<j and j<a and a<b and b<c and c<d and d<e and e<f and f<g and g<h:
                    #if i==4 and j==7 and a==14:

                      X = df.values[:, [i,j,a,b,c,d,e,f,g,h]]
                      who = "area " + str(i) + " - " + str(j) + " - " + str(a) + " - " + str(b) + " - " + str(c) + " - " + str(d) + " - " + str(e) + " - " + str(f) + " - " + str(g) + " - " + str(h)

                      X_train = X[:split_at]
                      X_test = X[split_at:]

                      y_train = y[:split_at]
                      y_test = y[split_at:]

                      # addestramento
                      #lr = LogisticRegression(C=100.0, random_state=0, class_weight='balanced') #parametro C è l'inverso della regolarizzazione; pertanto, se riduco il valore C, incremento la regolarizzazione; e viceversa
                      lr = LogisticRegressionCV(cv=5, random_state=0, class_weight='balanced') #scoring per cambiare metrica
                      lr.fit(X_train, y_train)
                      acc_s = lr.score(X_train, y_train)*100
                      acc_s_test = lr.score(X_test, y_test)*100

                      #if acc_s+acc_s_test > 150:
                      #print( who + ";" + "{:.1f}".format(acc_s) + ";" + "{:.1f}".format(acc_s_test) +";" + "{:.1f}".format(acc_s+acc_s_test) )
                      #print( conf_mat )

                      if max_accuracy_score < acc_s+acc_s_test:
                        max_accuracy_score = acc_s+acc_s_test
                        max_accuracy_score_who = who
                        print( who + ";" + "{:.1f}".format(acc_s) + ";" + "{:.1f}".format(acc_s_test) +";" + "{:.1f}".format(acc_s+acc_s_test) )



#print( max_f1_score_who )
print( "---------------------> " + max_accuracy_score_who + " " + str(max_accuracy_score) )
#print( conf_mat )



area 0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9;63.3;42.9;106.1
area 0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 9 - 12;62.2;45.2;107.5
area 0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 9 - 13;67.3;42.9;110.2
area 0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 10 - 13;75.5;57.1;132.7
area 0 - 1 - 2 - 3 - 4 - 5 - 6 - 9 - 12 - 13;75.5;59.5;135.0
area 0 - 1 - 2 - 3 - 4 - 6 - 7 - 10 - 12 - 13;77.6;64.3;141.8
area 0 - 1 - 2 - 4 - 5 - 6 - 7 - 8 - 10 - 13;77.6;66.7;144.2
area 0 - 2 - 3 - 4 - 5 - 6 - 7 - 10 - 12 - 13;74.5;71.4;145.9
area 0 - 2 - 3 - 4 - 6 - 7 - 9 - 10 - 12 - 13;77.6;71.4;149.0
area 0 - 2 - 4 - 5 - 6 - 7 - 10 - 11 - 12 - 13;74.5;76.2;150.7
